## San Francisco Crime Classification 

목차:

**[I. 데이터 불러오기 및 확인](#one)**


**[II. EDA & FE](#two)**

- [a. concat](#two-a)
- [b. 타겟변수 확인](#two-b)
- [c. Dates](#two-c)
- [d. PdDistrict](#two-d)
- [e. DayOfWeek](#two-e)
- [f. Address](#two-f)
- [g. X,Y](#two-g)
    
**[III. 모델링](#three)**

- [a. lightgbm](#three-a)
- [b. xgboost](#three-b)
- [c. 앙상블](#three-c)

## I. 데이터 불러오기 및 확인 <a id="one"></a>

1. 변수 | 설명
------- | -------
**Dates ** | 범죄가 일어난 날
**Category ** | 범죄 유형 
**Descript ** | 범죄에 대한 자세한 설명 
**DayOfWeek ** | 요일  
**PdDistrict ** | 경찰 관할 지역 명칭 
**Resolution ** | 범죄 해결 여부 
**Address ** | 범죄 발생 주소 
**X ** | 경도(Longitude) 
**Y** | 위도(Latitude)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train = pd.read_csv("../input/sf-crime/train.csv.zip", parse_dates=['Dates'])
test = pd.read_csv("../input/sf-crime/test.csv.zip", parse_dates=['Dates'], index_col='Id')
sampleSubmission = pd.read_csv("../input/sf-crime/sampleSubmission.csv.zip")

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
train.info()

> 결측치는 없는것으로 확인 되었습니다

### a.concat <a id="two-a"></a>

In [ ]:
fe_name = list(test)
df_train = train[fe_name]
df = pd.concat((df_train,test))

### b.타겟변수 확인 <a id="two-b"></a>

In [ ]:
target = train['Category']

In [ ]:
target.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder

LB = LabelEncoder() 
target = LB.fit_transform(target)
print(LB.classes_)

In [ ]:
target

> 데이터 모델링을 위해서 타겟변수를 라벨인코더 해줍니다

### c.Dates <a id="two-c"></a>

In [ ]:
date = pd.to_datetime(df['Dates'])
df['Date'] = date.dt.date
df['Year'] = date.dt.year
df['Month'] = date.dt.month
df['Day'] = date.dt.day
df['Hour'] = date.dt.hour

In [ ]:
df.drop("Dates", axis = 1, inplace = True)

In [ ]:
year = df.groupby('Year').count().iloc[:,0]
month = df.groupby('Month').count().iloc[:,0]
hour = df.groupby('Hour').count().iloc[:,0]
dayofweek = df.groupby('DayOfWeek').count().iloc[:, 0]

figure, axs = plt.subplots(2,2, figsize = (15,10))

sns.barplot(x=year.index, y= year,ax = axs[0][0])
sns.barplot(x=month.index, y= month,ax = axs[0][1])
sns.barplot(x=hour.index, y= hour,ax = axs[1][0])
sns.barplot(x=dayofweek.index, y= dayofweek,ax = axs[1][1])

>연도에 따른 범죄수 확인 결과  2015년에는 급격히 떨어진것을 확인해볼 수 있습니다.
>
>시간에 따른 범죄수 확인 결과 새벽시간에 제일 적었으며 12,18시가 가장 많은 것으로 보입니다.

In [ ]:
date = df.groupby('Date').count().iloc[:, 0]

In [ ]:
sns.kdeplot(data=date, shade=True)
plt.axvline(x=date.median(), ymax=0.95, linestyle='--')
plt.annotate('Median ' + str(date.median()),xy =(date.median(), 0.005))

> 하루 동안 발생하는 범죄수는 정규분포를 보이고 있으며 그 중앙값은 389입니다.

In [ ]:
lb = LabelEncoder()
df['PdDistrict'] = lb.fit_transform(df["PdDistrict"])

### d.PdDistrict <a id="two-d"></a>

In [ ]:
df["PdDistrict"].value_counts()

In [ ]:
sns.countplot(df["PdDistrict"])

In [ ]:
lb = LabelEncoder()
df['PdDistrict'] = lb.fit_transform(df["PdDistrict"])

> PdDistrict 피처에 대해서 라벨 인코더 해줍니다

### e.DayOfWeek <a id="two-e"></a>

In [ ]:
df['DayOfWeek'] = lb.fit_transform(df["DayOfWeek"])

> DayOfWeek 피처에 대해서 라벨 인코더 해줍니다

### f.Address <a id="two-f"></a>

In [ ]:
df["Address"].value_counts().head(20)

In [ ]:
df['block'] = df['Address'].str.contains('block', case=False)
df['ST'] = df['Address'].str.contains('ST', case=False)

In [ ]:
df['block'] = lb.fit_transform(df["block"])
df['ST'] = lb.fit_transform(df["ST"])

In [ ]:
df.drop("Address", axis = 1, inplace = True)

> Address 피처에서 block과 ST 즉 구역과 거리를 포함하고 있는지 없는지를 구분해주는 새로운 파생변수를 만들어 줍니다

### g.X,Y <a id="two-g"></a>

In [ ]:
print(df["X"].min(), df["X"].max())
print(df["Y"].min(), df["Y"].max())

In [ ]:
print(len(df.loc[df["X"] >= -120.5, "X"]))
print(len(df.loc[df["Y"] >= 90, "Y"]))

In [ ]:
X_median = df[df["X"] < -120.5]["X"].median()
Y_median = df[df["Y"] < 90]["Y"].median()
df.loc[df["X"] >= -120.5, "X"] = X_median
df.loc[df["Y"] >= 90, "Y"] = Y_median

>143개의 X와 Y의 값이 잘못 표기되어 있는 것으로 보아 중앙값으로 대체해 줍니다.

In [ ]:
df["X+Y"] = df["X"] + df["Y"]
df["X-Y"] = df["X"] - df["Y"]

In [ ]:
df.drop("Date", axis = 1, inplace = True)

> 위도와 경도의 합과 차이를 구하는 새로운 파생 변수를 만들어 줍니다

## III. 모델링 <a id="three"></a>

In [ ]:
new_train = df[:train.shape[0]]
new_test = df[train.shape[0]:]

In [ ]:
new_train.head()

### a. lightgbm <a id="three-a"></a>

In [ ]:
import lightgbm as lgb

train_data = lgb.Dataset(new_train, label=target, categorical_feature=["PdDistrict", "DayOfWeek"])
params = {'boosting':'gbdt',
          'objective':'multiclass',
          'num_class':39,
          'max_delta_step':0.9,
          'min_data_in_leaf': 21,
          'learning_rate': 0.4,
          'max_bin': 465,
          'num_leaves': 41,
          'verbose' : 1}
bst = lgb.train(params, train_data, 120)

In [ ]:
predictions = bst.predict(new_test)

In [ ]:
submission = pd.DataFrame(predictions,columns=LB.inverse_transform(np.linspace(0, 38, 39, dtype='int16')),index=new_test.index)
#submission.to_csv('LGB.csv', index_label='Id')

> Yannis Pappas의 커널에서 LGBM 모델의 파라미터를 참고하였습니다. Bayesian Optimization을 통해서 하이퍼 파라미터 튜닝을 해줘야 하지만 추후에 하이퍼 파라미터 튜닝을 해줘야겠습니다.

### b. xgboost <a id="three-b"></a>

In [ ]:
import xgboost as xgb
train_xgb = xgb.DMatrix(new_train, label=target)
test_xgb  = xgb.DMatrix(new_test)

In [ ]:
params = {
    'max_depth': 4,  
    'eta': 0.3,  
    'silent': 1, 
    'objective': 'multi:softprob', 
    'num_class': 39,
}

xg = xgb.cv(params, train_xgb, nfold=3, early_stopping_rounds=10, metrics='mlogloss', verbose_eval=True) 

In [ ]:
train_xgb = xgb.train(params, train_xgb, 10)
pred_xgb = train_xgb.predict(test_xgb)

In [ ]:
submission1 = pd.DataFrame(pred_xgb,columns=LB.inverse_transform(np.linspace(0, 38, 39, dtype='int16')),index=new_test.index)
#submission1.to_csv('XGB.csv', index_label='Id')

### c. 앙상블 <a id="three-c"></a>

In [ ]:
ensemble = 0.9*predictions + 0.1*pred_xgb

In [ ]:
sub = pd.DataFrame(ensemble,columns=LB.inverse_transform(np.linspace(0, 38, 39, dtype='int16')),index=new_test.index)
sub.to_csv('submission.csv', index_label='Id')